In [1]:
import sys

sys.path.append('/home/alta/Conversational/OET/al826/2022/conv_frameworks/parallel_exp')

In [2]:
from src.eval.integrad_handler import IntegradHandler

E = IntegradHandler('sep_focus_utt/full', hpc=True)
#E.parallelise()

Model found!


In [6]:
from types import SimpleNamespace

eval_path = f"swda/standard/dev.json"

test_args = {'eval_path':eval_path,
             'bsz':1,
             'lim':None}

test_args = SimpleNamespace(**test_args)

conv_num = 12
output = E.conv_integrad(test_args, conv_num=conv_num, N=100)
E.dir.save_dict(f'integrad/{conv_num}', output)


100%|██████████| 256/256 [8:52:24<00:00, 124.78s/it]  


In [4]:
def get_avg_dist(utt, utt_num):
    def normalise(scores):
        scores = [abs(i) for i in scores]
        scores = [i/sum(scores) for i in scores]
        return scores

    utt_scores = utt['utt_scores']
    
    #filter out scores < 1e-3
    utt_scores = [a if abs(a) > 1e-3 else 0 for a in utt_scores]
    
    #normalise
    utt_scores = normalise(utt_scores)
    
    avg_dist = sum([a*abs(utt_num-k) for k, a in enumerate(utt_scores)])
    return avg_dist

x = [get_avg_dist(output[k], k) for k in range(5)]
print(x)

[7.040592706239434, 2.8375988565669026, 8.03145440756653, 5.488012782047294, 4.995244306525801]


In [5]:
x = [i for j in output for i in j]
print([i['true_class'] for i in x])

TypeError: string indices must be integers

['statement', 'statement', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'backchannel', 'opinion', 'backchannel', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'backchannel', 'backchannel', 'opinion', 'opinion', 'backchannel', 'multi_segment', 'opinion', 'backchannel', 'opinion', 'backchannel', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'agreement', 'opinion', 'abandoned', 'opinion', 'agreement', 'opinion', 'backchannel', 'opinion', 'backchannel', 'hedge', 'backchannel', 'opinion', 'opinion', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'non verbal', 'non verbal', 'appreciation', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'abandoned', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'statement', 'abandoned', 'opinion', 'backchannel', 'multi_segment', 'abandoned', 'multi_segment', 'statement', 'backchannel', 'multi_segment', 'opinion', 'backchannel', 'multi_segment', 'agreement', 'opinion', 'opinion', 'backchannel', 'abandoned', 'statement', 'backchannel', 'statement', 'backchannel', 'statement', 'statement', 'statement', 'backchannel', 'multi_segment', 'statement', 'backchannel', 'multi_segment', 'backchannel', 'abandoned', 'statement', 'statement', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'backchannel', 'backchannel', 'statement', 'backchannel', 'abandoned', 'multi_segment', 'backchannel', 'multi_segment', 'statement', 'statement', 'opinion', 'backchannel', 'opinion', 'backchannel', 'multi_segment', 'backchannel', 'multi_segment', 'backchannel', 'statement', 'statement', 'backchannel', 'statement', 'backchannel', 'multi_segment']

In [ ]:
from types import SimpleNamespace

eval_path = f"swda/standard/test.json"

test_args = {'eval_path':eval_path,
             'bsz':2,
             'lim':None}

test_args = SimpleNamespace(**test_args)

words, scores = E.saliency(test_args, N=20, utt_num=31)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

words = [i.replace('<s>', '[CLS]') for i in words]
words = [i.replace('</s>', '[SEP]') for i in words]
s = [0] + [k for k, word in enumerate(words) if word == '[SEP]']
turn_scores = [sum(scores[s[i]:s[i+1]]) for i in range(len(s)-1)]

#temp
plt.rcParams["figure.figsize"] = 10,1

dist = [abs(i) for i in turn_scores]
dist = np.array(dist)/sum(dist)
plt.imshow(dist[np.newaxis,:], cmap="plasma", aspect="auto", vmin=0, vmax=1)
plt.show()
